# US Counties Coronavirus Cases and Deaths Trends

CA's [Blueprint for a Safer Economy](https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyMonitoringOverview.aspx) assigns each county [to a tier](https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyMonitoringOverview.aspx) based on case rate and test positivity rate. What's opened / closed [under each tier](https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH%20Document%20Library/COVID-19/Dimmer-Framework-September_2020.pdf).

Tiers, from most severe to least severe, categorizes coronavirus spread as <strong><span style='color:#6B1F84'>widespread; </span></strong>
<strong><span style='color:#F3324C'>substantial; </span></strong><strong><span style='color:#F7AE1D'>moderate; </span></strong><strong><span style = 'color:#D0E700'>or minimal.</span></strong>

We apply CA's tier categories to other US counties to help us understand the severity of the spread. 

**Related daily reports:** 
1. **[CA counties report on cases, deaths, and hospitalizations](https://cityoflosangeles.github.io/covid19-indicators/ca-county-trends.html)**
1. **[Los Angeles County, detailed indicators](https://cityoflosangeles.github.io/covid19-indicators/coronavirus-stats.html)**
1. **[Los Angeles County neighborhoods report on cases and deaths](https://cityoflosangeles.github.io/covid19-indicators/la-neighborhoods-trends.html)**

Code available in GitHub: [https://github.com/CityOfLosAngeles/covid19-indicators](https://github.com/CityOfLosAngeles/covid19-indicators)
<br>
Get informed with [public health research](https://github.com/CityOfLosAngeles/covid19-indicators/blob/master/reopening-sources.md)

In [ ]:
import altair as alt
import altair_saver
import os
import pandas as pd

import default_parameters
import make_charts
import us_county_utils

from IPython.display import display_html, Markdown, HTML, Image

# Default parameters
time_zone = default_parameters.time_zone
start_date = default_parameters.start_date
today_date = default_parameters.today_date

fulldate_format = default_parameters.fulldate_format

In [ ]:
northeast = [
    "Suffolk, MA", # Boston
    "New York, NY", 
    "Philadelphia, PA", 
    "Allegheny, PA", # Pittsburg
    
]

midatlantic = [
    "District of Columbia, DC", 
    "Arlington, VA", 
    "Fairfax, VA",
    "Richmond City, VA", 
]

midwest = [
    "Cook, IL", 
    "Dane, WI", # Madison 
    "Hamilton, OH", # Cincinnati
    "Fayette, KY", # Lexington
    "Hennepin, MN", # Minneapolis
    "St. Louis, MO",
]

southeast = [
    "Mecklenburg, NC", # Charlotte
    "Richland, SC", # Columbia
    "Shelby, TN", # Memphis
    "Davidson, TN", # Nashville
    "Knox, TN", # Knoxville
    "Fulton, GA", # Atlanta
    "Orange, FL", # Orlando
]

south = [
    "Travis, TX", # Austin
    "Dallas, TX", 
    "Harris, TX", # Houston
    "Oklahoma, OK", # Oklahoma City
]

northwest = [
    "Multnomah, OR", # Portland
    "King, WA", # Seattle
]

west = [
    "Maricopa, AZ", # Phoenix
    "Clark, NV", # Las Vegas
    "Salt Lake, UT", # Salt Lake City
    "Denver, CO",
    "Bernalillo, NM", # Alberquerque
]

# Put all states and counties into list
include_counties = northeast + midatlantic + midwest + southeast + south + west + northwest

# Map counties to major cities
major_city_dict = {
    # Northeast
    'Suffolk, MA': 'Boston',
    'New York, NY': 'New York City',
    'Philadelphia, PA': 'Philadelphia',
    'Allegheny, PA': 'Pittsburgh',
    # Mid-Atlantic
    'District of Columbia, DC': 'Washington DC',
    'Arlington, VA': 'Arlington',
    'Fairfax, VA': 'Fairfax',
    'Richmond City, VA': 'Richmond',
    # Midwest
    'Cook, IL': 'Chicago',
    'Dane, WI': 'Madison',
    'Hamilton, OH': 'Cincinnati',
    'Fayette, KY': 'Lexington',
    'Hennepin, MN': 'Minneapolis',
    'St. Louis, MO': 'St Louis',
    # Southeast
    'Mecklenburg, NC': 'Charlotte',
    'Richland, SC': 'Columbia',
    'Shelby, TN': 'Memphis',
    'Davidson, TN': 'Nashville',
    'Knox, TN': 'Knoxville',
    'Fulton, GA': 'Atlanta',
    'Orange, FL': 'Orlando',
    # South
    'Travis, TX': 'Austin',
    'Dallas, TX': 'Dallas', 
    'Harris, TX': 'Houston',
    'Oklahoma, OK': 'Oklahoma City',
    # Northwest / West
    'Multnomah, OR': 'Portland',
    'King, WA': 'Seattle',
    'Maricopa, AZ': 'Phoenix',
    'Clark, NV': 'Las Vegas',
    'Salt Lake, UT': 'Salt Lake City',
    'Denver, CO': 'Denver',
    'Bernalillo, NM': 'Alburquerque',
}

In [ ]:
jhu = us_county_utils.clean_jhu(start_date)

jhu = jhu.assign(
    county_state = jhu.county + ", " + jhu.state_abbrev
)

jhu = (jhu[jhu.county_state.isin(include_counties)]
      .assign(
          major_city = jhu.county_state.map(major_city_dict)
      )
     )

data_through = jhu.date.max()

In [ ]:
display(Markdown(
    f"Report updated: {default_parameters.today_date.strftime(fulldate_format)}; "
    f"data available through {data_through.strftime(fulldate_format)}")
)

In [ ]:
title_font_size = 9
font_name = make_charts.font_name
grid_opacity = make_charts.grid_opacity
domain_opacity = make_charts.domain_opacity
stroke_opacity = make_charts.stroke_opacity
chart_width = make_charts.chart_width
chart_height = make_charts.chart_height


def plot_charts(cases_df, county_state_name):
    cases_df = cases_df[cases_df.county_state==county_state_name]
    
    name = cases_df.county.iloc[0]
    STATE = cases_df.state_abbrev.iloc[0]
    
    cases_chart, deaths_chart = make_charts.setup_cases_deaths_chart(cases_df, "county", name)
    
    chart = (alt.hconcat(cases_chart, deaths_chart)
             .configure_title(
                 fontSize=title_font_size, font=font_name, anchor="middle", color="black"
             ).configure_axis(
                 gridOpacity=grid_opacity, domainOpacity=domain_opacity
             ).configure_view(strokeOpacity=stroke_opacity)
            )
    
    major_city_name = cases_df.major_city.iloc[0]
    display(Markdown(f"#### {major_city_name} ({county_state_name})"))
    us_county_utils.county_caption(cases_df, name)

    make_charts.show_svg(chart)

In [ ]:
display(Markdown("<strong>Cases chart, explained</strong>"))
Image("../notebooks/chart_parts_explained.png", width=700)

<a id='top'></a>

## Select by Major City
<strong>Northeast:</strong> [Boston](#Boston), 
[New York City](#New-York-City), 
[Philadelphia](#Philadelphia), 
[Pittsburgh](#Pittsburgh) 
<br>
<strong>Mid-Atlantic: </strong> [Washington DC](#Washington-DC), 
[Arlington](#Arlington),
[Fairfax](#Fairfax),
[Richmond](#Richmond)
<br>
<strong>Midwest:</strong> [Chicago](#Chicago), 
[Madison](#Madison),
[Cincinnati](#Cincinnati), 
[Lexington](#Lexington),
[Minneapolis](#Minneapolis),
[St Louis](#St-Louis)
<br>
<strong>Southeast:</strong> [Charlotte](#Charlotte), 
[Columbia](#Columbia), 
[Memphis](#Memphis), 
[Nashville](#Nashville),
[Knoxville](#Knoxville),
[Atlanta](#Atlanta),
[Orlando](#Orlando)
<br>
<strong>South: </strong> [Austin](#Austin),
[Dallas](#Dallas),
[Houston](#Houston),
[Oklahoma City](#Oklahoma-City)
<br>
<strong>West / Northwest:</strong> [Portland](#Portland), 
[Seattle](#Seattle),
[Phoenix](#Phoenix),
[Las Vegas](#Las-Vegas),
[Salt Lake City](#Salt-Lake-City),
[Denver](#Denver),
[Alburquerque](#Alburquerque)
<br>
<strong>California:</strong> [All CA counties](https://cityoflosangeles.github.io/covid19-indicators/ca-county-trends.html)

<br>
Note for <i>small values</i>: If the 7-day rolling average of new cases or new deaths is under 10, the 7-day rolling average is listed for the past week, rather than a percent change. Given that it is a rolling average, decimals are possible, and are rounded to 1 decimal place. 

In [ ]:
def plot_regional_charts(regional_counties_list):
    for c in region:
        # Map the county_state name into major city
        new_name = major_city_dict[c]

        # Set the HTML id_anchor
        id_anchor = new_name.replace(" - ", "-").replace(" ", "-")
        display(HTML(f"<a id={id_anchor}></a>"))
        
        plot_charts(jhu, c)    
        display(HTML(
            "<br>"
            "<a href=#top>Return to top</a><br>"
        ))

## Northeast

In [ ]:
region = northeast
plot_regional_charts(region)

## Mid-Atlantic

In [ ]:
region = midatlantic
plot_regional_charts(region)

## Midwest

In [ ]:
region = midwest
plot_regional_charts(region)

## Southeast

In [ ]:
region = southeast
plot_regional_charts(region)

## South

In [ ]:
region = south
plot_regional_charts(region)

## Northwest / West

In [ ]:
region = northwest
plot_regional_charts(region)

In [ ]:
region = west
plot_regional_charts(region)

<a id='dc'></a>